# Advanced testing

Up till now we have been testing functions where the output is entirely predictable. In these cases, a handful of tests is usually enough to provide confidence that the software is working as expected. In the real world, however, you might be developing a complex piece of sofware to implement an entirely new algorithm, or model. In certain cases it might not even be clear what the expected outcome is meant to be. Things can be particularly challenging when the software is involves a stochastic element.

Let us consider a class to simulate the behaviour of a dice. One is provided in the `dice` directory. Let's import it and see how it works.

In [1]:
from dice import Dice
help(Dice)

Help on class Dice in module dice.dice:

class Dice(builtins.object)
 |  A simple class for an n-sided fair dice.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, n=6, seed=None)
 |      Construct a n-sided dice.
 |      
 |      n -- The number of sides on the dice.
 |  
 |  lastRoll(self)
 |      Return the value of the last dice roll.
 |  
 |  roll(self)
 |      Roll the dice and return its value.
 |  
 |  sides(self)
 |      Return number of sides of the dice.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



How could we test that the dice is fair?

Well, first of all we could test that the value of a dice roll is in range.

In [2]:
dice = Dice()        # A standard, six-sided dice.
roll = dice.roll()
assert roll > 0
assert roll < 7

Great, that worked. Although, it could just be a fluke...

In practice, we need to check that the assertions hold repeatedly.

In [3]:
for i in range(0, 1000):
    roll = dice.roll()
    assert roll > 0
    assert roll < 7

Okay, that's better. Or is it...

![xkcd: random](https://imgs.xkcd.com/comics/random_number.png)

Not again!

Perhaps we should test the average value. We know that this should equal the sum of the faces of the dice, divided by the number of sides, i.e. 3.5 for a six-sided dice.

In [4]:
from pytest import approx

rolls = 1000000
sum = 0
for i in range(0, rolls):
    sum += dice.roll()

sum /= rolls

assert sum == approx(3.5, 1e-3)

Good... Hang on, hold your horses!

In [5]:
(1 + 3 + 4 + 6) / 4

3.5

Dang! We need to test that the _distrubtion_ of outcomes is correct, i.e. that each of the six possible outcomes is equally likely.

In [6]:
rolls = 1000000

tally = {}
for i in range(1, 7):
    tally[i] = 0
    
for i in range(0, rolls):
    tally[dice.roll()] += 1

for i in range(1, 7):
    assert tally[i] / rolls == approx(1/6, 1e-2)

Phew, thanks goodness! Testing is hard.

# Exercise

#### Exercise 1

The file `dice/test/test_dice.py` contains an empty function, `test_double_roll`, for checking that the distribution for the sum of two dice rolls is correct. Fill in the body of this function to test that the distrution is correct for two standard six-sided dice. Run `pytest dice` to verify that your test passes.

_Hints_:

For any two n-sided dice, the probability of the sum of two rolls being a value of `x` is given by:

$$p(x) = \frac{n - |x - (n+1)|}{n^2},\quad\mathrm{for}\ x=2\ \mathrm{to}\ 2n$$

In python, the absolute value of a number can be found using the `abs` function, e.g.

```python
abs(-2) == 2
True
```

#### Exercise 2

Parametrize your test function so that it works for any pair of n-sided dice.